Con LlamaIndex no pude ingestar. 4 NOV 21:10 probndo con langchain y unstructured

In [1]:
from langchain.document_loaders import UnstructuredFileLoader

In [2]:
loader = UnstructuredFileLoader("../../data/mastercard/s3.pdf")

In [3]:
docs = loader.load()

In [13]:
type(docs[0])

langchain.schema.document.Document

In [15]:
docs[0].metadata

{'source': '../../data/mastercard/s3.pdf'}

In [7]:
docs[0].page_content

'Resumen N° 027051366624\n\nTarjeta Crédito MASTERCARD PLATINUM\n\nDITZEND,ALEXANDER\n\nCONSUMIDOR FINAL\n\nLADINES 2646 3, CIUDAD AUTONOMA BUEN, 01419\n\nN° de Socio: 1417647-0-0\n\n102.041,04\n\n07-Sep-23\n\n15-Sep-23\n\n05-Oct-23\n\n17-Oct-23\n\n7,53\n\nPAGO MINIMO En pesos\n\nLÍMITES\n\nDe compra\n\nEn cuotas\n\n$ 9.520,00\n\n$ 128.000,00\n\n$ 128.000,00\n\nTASAS\n\nNominal Anual\n\nEn pesos 106,640%\n\nEn dólares 0,000%\n\nEfectiva mensual\n\nEn pesos 8,740%\n\nCONSOLIDADO\n\nSALDO ANTERIOR\n\n11-Sep-23\n\nSU PAGO\n\n113.162,38\n\nSALDO PENDIENTE\n\nTOTAL CONSUMOS DEL MES\n\nSUBTOTAL\n\nINTERESES DE FINANCIACION\n\nIMPUESTO DE SELLOS\n\nI.V.A. 21,0%\n\nIMPUESTO PAIS\n\nPERCEP.AFIP RG 4815 45%\n\nTOTAL A PAGAR\n\nDETALLE DEL CONSUMO\n\nFECHA\n\nREFERENCIA\n\nCOMPROBANTE\n\nCOMPRAS DEL MES\n\n24-Sep-23\n\nSCAPE PARK (DOM,ARS, 2603,00)\n\n00997\n\n08-Sep-23\n\nAUTOPISTAS DEL 10/18\n\n02310\n\n14-Sep-23\n\nDON WEB\n\n07013\n\n14-Sep-23\n\nDON WEB\n\n07014\n\n22-Sep-23\n\nPPRO *Adobe\n

In [28]:
import requests
import os
import json
def gpt_system_user(
    system_message: str, user_message: str, model: str = "gpt-3.5-turbo"
):
    """Para usar en notebooks"""
    try:
        response = requests.post(
            url="https://api.openai.com/v1/chat/completions",
            params={
                "temperature": "0",
            },
            headers={
                "Content-Type": "application/json",
                "Authorization": f"Bearer {os.environ['OPENAI_API_KEY']}",
            },
            data=json.dumps(
                {
                    "model": model,
                    "messages": [
                        {"content": system_message, "role": "system"},
                        {"content": user_message, "role": "user"},
                    ]
                }
            ),
            timeout=10,
        )
        choices = response.json()["choices"]
        answer = {
            "status_code": response.status_code,
            "text": choices[0]["message"],
        }
        return answer

    except requests.exceptions.RequestException:
        print("HTTP Request failed")
        return None


Nuevo modelo instruct  `gpt-3.5-turbo-instruct-0914`

In [42]:
import requests
import os
import json
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

def gpt_instruct(
    instruction: str, model: str = "gpt-3.5-turbo-instruct"
):
    """Para usar en notebooks"""
    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello!"}
      ]
    )

  print(completion.choices[0].message)
    return model_response["choices"][0]["text"]


In [43]:
context = docs[0].page_content
instruction = f"""
Extract all meaningful information from the document between three backticks.
This is very important information for the user, so please be sure to include all the relevant information.
Use this format:
"The user [did/bought/used/paid/...] [something] on [date] at [place/store] for [amount of money]."
"The total amount of money spent in the month was [amount of money]."

```
{context}
```
"""

summary = gpt_instruct(instruction=instruction)
summary

'"The user [used] their MASTERCARD PLATINUM credit card'

Instruct genera un resumen horrible. No sirve para nada.

Pruebo con turbo.

In [44]:
context = docs[0].page_content
system_message = f"""
The user will give you an important document with some meaningful data surrounded by meaningless data.
Extract all meaningful information from the document and create a summary.
No important details should be left out.
This is very important information for the user, so please be sure to include all the relevant information.
Use this format:
"The user [did/bought/used/paid/...] [something] on [date] at [place/store] for [amount of money]."
"""

chat_summary = gpt_system_user(system_message=system_message, user_message=context)
chat_summary['text']

{'status_code': 200,
 'text': {'role': 'assistant',
  'content': 'The user used their MASTERCARD PLATINUM credit card for various purchases in the month of September. Here are the details of the purchases:\n\n- On September 24, they made a purchase for 2603,00 ARS at SCAPE PARK.\n- On September 8, they made a purchase at AUTOPISTAS DEL 10/18.\n- On September 14, they made two purchases at DON WEB.\n- On September 22, they made a purchase at PPRO *Adobe.\n\nThe total amount to be paid is 102,041.04 ARS, with a minimum payment of 7.53 ARS. \n\nThe card also has the following limits:\n- Purchase limit: 9,520.00 ARS\n- Installment limit: 128,000.00 ARS\n\nThe card has a previous balance of 113,162.38 ARS and a pending balance of 126,372.38 ARS.\n\nThe user has until November 9 to make the payment.\n\nThe interest rate for financing in pesos is 106.640% nominal annual rate and 8.740% effective monthly rate.\n\nThe card also has a cash advance option with a 107.000% nominal annual rate.\n\nT

In [45]:
summary = chat_summary['text']['content']

In [46]:
summary

'The user used their MASTERCARD PLATINUM credit card for various purchases in the month of September. Here are the details of the purchases:\n\n- On September 24, they made a purchase for 2603,00 ARS at SCAPE PARK.\n- On September 8, they made a purchase at AUTOPISTAS DEL 10/18.\n- On September 14, they made two purchases at DON WEB.\n- On September 22, they made a purchase at PPRO *Adobe.\n\nThe total amount to be paid is 102,041.04 ARS, with a minimum payment of 7.53 ARS. \n\nThe card also has the following limits:\n- Purchase limit: 9,520.00 ARS\n- Installment limit: 128,000.00 ARS\n\nThe card has a previous balance of 113,162.38 ARS and a pending balance of 126,372.38 ARS.\n\nThe user has until November 9 to make the payment.\n\nThe interest rate for financing in pesos is 106.640% nominal annual rate and 8.740% effective monthly rate.\n\nThe card also has a cash advance option with a 107.000% nominal annual rate.\n\nThe card is associated with a Banco account with CUIT 30-50000173-